In [4]:
import torch
import torch.nn as nn
import torchvision

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())#输出false，表示当前pytorch无法使用显卡

1.11.0+cpu
None
None
False


In [7]:
# 在同一个设备上，应该保证可复现性。具体做法是，在程序开始的时候固定torch的随机种子，同时也把numpy的随机种子固定。
import numpy as np

np.random.seed(0)
torch.manual_seed(0)#设置随机数种子
torch.cuda.manual_seed_all(0)#多个GPU时，为所有GPU设置种子

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [8]:
# 显卡设置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
# torch.cuda.empty_cache()

cpu


In [9]:
#张量处理
tensor = torch.randn(3,4,5)
print(tensor.type())
print(tensor.size())
print(tensor.dim())

torch.FloatTensor
torch.Size([3, 4, 5])
3


In [11]:
# 张量命名
NCHW = ['N','C','H','W']
images = torch.randn(32, 3, 56, 56, names=NCHW)
images.sum('C')
images.select('C', index=0)

tensor([[[-9.9507e-01,  4.5089e-01,  1.4671e-01,  ...,  3.4687e-01,
          -2.9055e-01, -1.7921e+00],
         [-5.8186e-01, -2.5670e-01,  7.5862e-01,  ..., -3.1995e-02,
          -4.1721e-01, -7.8874e-01],
         [ 1.1152e+00, -1.1732e+00,  1.7059e-01,  ...,  9.0192e-01,
           9.1724e-01,  8.2328e-01],
         ...,
         [ 1.2963e+00, -9.4490e-01,  5.7915e-01,  ..., -8.5010e-01,
          -1.0993e+00,  8.6259e-01],
         [-2.2042e-01,  3.9036e-01, -9.8582e-02,  ..., -1.8150e-01,
           9.1908e-02,  4.3104e-02],
         [ 1.9219e+00,  2.1617e+00, -6.1160e-01,  ...,  1.5528e-01,
          -1.9943e+00, -1.0316e+00]],

        [[-2.3882e+00,  1.0923e+00,  6.3999e-01,  ..., -9.5147e-01,
           4.0850e-02,  2.7926e-01],
         [-7.1295e-01,  1.1391e-01,  5.9903e-01,  ...,  1.7090e-01,
           5.9664e-01, -3.2352e-01],
         [-1.7379e+00, -1.2598e+00,  2.7185e-01,  ..., -1.0333e+00,
          -2.4391e-01, -2.1693e-01],
         ...,
         [ 8.2945e-01, -1

In [14]:
# 设置默认类型，pytorch中的FloatTensor远远快于DoubleTensor
torch.set_default_tensor_type(torch.FloatTensor)

# 类型转换
# tensor = tensor.cuda()
tensor = tensor.cpu()
# tensor = tensor.float()
# tensor = tensor.long()

In [17]:
# tensor和ndarray互换
# tensor用逗号隔开，ndarray用空格隔开
ndarray = tensor.cpu().numpy()
tensor = torch.from_numpy(ndarray).float()
print(ndarray,'\n',tensor)

[[[-1.1258398  -1.1523602  -0.25057858 -0.4338788   0.84871036]
  [ 0.69200915 -0.31601277 -2.1152194   0.32227492 -1.2633348 ]
  [ 0.3499832   0.30813393  0.11984151  1.2376579   1.1167772 ]
  [-0.24727815 -1.3526537  -1.6959312   0.5666506   0.79350835]]

 [[ 0.59883946 -1.5550951  -0.3413604   1.8530061   0.7501895 ]
  [-0.58549756 -0.17339675  0.18347794  1.3893661   1.5863342 ]
  [ 0.94629836 -0.84367675 -0.6135831   0.03159274 -0.49267697]
  [ 0.24841475  0.43969584  0.11241119  0.64079237  0.44115627]]

 [[-0.10230965  0.792444   -0.2896677   0.05250749  0.59432304]
  [ 1.5419426   0.5073344  -0.59103316 -0.5692481   0.9199714 ]
  [ 1.1108161   1.2898741  -1.4959149  -0.19383712  0.4455122 ]
  [ 1.3252748  -1.629326   -0.54974365 -0.47983426 -0.49968153]]] 
 tensor([[[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
         [ 0.6920, -0.3160, -2.1152,  0.3223, -1.2633],
         [ 0.3500,  0.3081,  0.1198,  1.2377,  1.1168],
         [-0.2473, -1.3527, -1.6959,  0.5667,  0.7935]],

In [21]:
print(tensor[1,2,3])#取近似数
tensor[1,2,3].item()#取精确值

tensor(0.0316)


0.03159274160861969

In [29]:
# 张量连续性stride[i]=stride[i+1]*size[i+1]
# 相比torch.view，torch.reshape可以自动处理输入张量不连续的情况。
tensor = torch.rand(2,3,4)
print(tensor.size())
print(tensor.stride())
shape = (6, 4)
tensor = torch.reshape(tensor, shape)
print(tensor.size())
print(tensor.stride())

torch.Size([2, 3, 4])
(12, 4, 1)
torch.Size([6, 4])
(4, 1)


In [30]:
# 打乱顺序
# tensor = tensor[torch.randperm(tensor.size(0))]  # 打乱第一个维度
# 水平翻转
# pytorch不支持tensor[::-1]这样的负步长操作，水平翻转可以通过张量索引实现
# 假设张量的维度为[N, D, H, W].
tensor = tensor[:,:,:,torch.arange(tensor.size(3) - 1, -1, -1).long()]

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 3)

In [27]:
torch.arange(tensor.size(3) - 1, -1, -1).long()

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 3)